In [ ]:
import forward_run
import flopy
import os
import helpers
import matplotlib.pyplot as plt
import conda_scripts
import conda_scripts.load_datasets as lsd

In [ ]:
model_ws = lsd.model_info.get_mod('son', True)
model_ws = 'pestaws_lager'

In [ ]:
main = 'postprocess'
pstname =  'lager'
fold =  'lager'

out_folder = os.path.join(main, fold)
if not os.path.exists(out_folder):
    os.makedirs(out_folder, exist_ok =True)

In [ ]:
sr = helpers.get_sr()

In [ ]:
ml = conda_scripts.sv_budget.load_sv_model.get_model(model_ws)

In [ ]:
import numpy as np

In [ ]:
def _add_inset(masked_data, ax):
    # Ravel and remove NaNs
    ravelled_data = masked_data.ravel()
    ravelled_data = ravelled_data[~ravelled_data.mask]
    
    # Take the log of the values
    log_values = np.log10(ravelled_data)
    
    # Create the inset axes for the histogram
    inset_ax = ax.inset_axes([0.7, 0.7, 0.3, .3], transform=ax.transAxes)
    inset_ax.hist(log_values, bins=30, color='orange')
    # Set the y-axis labels to the right side of the inset axes
    inset_ax.yaxis.tick_right()

In [ ]:
prefix = forward_run.get_prefix_dict_for_pilot_points()
prefix

In [ ]:
import cartopy.crs as ccrs
import matplotlib.colorbar as colorbar
from matplotlib.colors import LogNorm

os.makedirs(os.path.join(out_folder,'array_explainer'), exist_ok = True)




for layer, props in prefix.items():
    for prop in props:
        
        ibound = ml.bas6.ibound.array[layer]
        
        
        print(f'{layer} {prop}')
        out, lay, mult, hk_arr,zmult =  forward_run.write_pilot_point(layer, prop, model_ws, skip_writing_output = True)
        
        d = {'Final Array':out, "Entire Zone":zmult, "Zone Multiplier":mult, "Pilot Point":hk_arr}

        if prop.lower()=='sy1':
            continue
            
        num = 1
        fig = plt.figure(figsize=(8.5,11))
        fig.suptitle(f"Layer {layer+1} {prop}")
        for key, array in d.items() :
            
            ax = plt.subplot(2, 2, num, projection=ccrs.epsg(2226))
            ax.set_title(key)
            # mm = conda_scripts.make_map.make_map(f"Layer {layer+1}, Aquifer Property: {prop}")
            # ax = mm.plotloc(fig, locname = 'SON_MOD')
            array = np.ma.masked_array(array, mask = ibound ==0)
            # Set up the normalization using LogNorm
            norm = LogNorm(vmin=10 ** np.floor(np.log10(array.min())),
                           vmax=10 ** np.ceil(np.log10(array.max())))
            c = ax.pcolormesh(sr.xcentergrid,sr.ycentergrid,array,alpha=0.5, norm = norm)
            conda_scripts.arich_functions.add_basemaps(ax)
            num+=1
            # ax.scatter(hk_pp2.x, hk_pp2.y,marker='o', c = hk_pp2.zone,s=25, cmap = 'tab10', ec = 'k')
        
            _add_inset(array, ax)

            # Create the inset axes for the histogram
            cax = ax.inset_axes([0.1, 0.005, .8, .03])
            # Add a colorbar at the bottom
            
            # Create a colorbar with the log normalization
            cbar = colorbar.ColorbarBase(cax, norm=norm, orientation='horizontal', cmap='viridis')
            # cbar = plt.colorbar(c, cax=cax, orientation='horizontal')
            
        print(f'{layer}_{prop}.png')
        plt.savefig(os.path.join(out_folder,'array_explainer',f'layer_{layer+1}_{prop}.png'))
        plt.close()
        

In [ ]:
out, lay, mult, hk_arr,zmult =  forward_run.write_pilot_point(4, 'hk6', model_ws, skip_writing_output = True)

In [ ]:
array = np.ma.masked_array(out, mask = ibound ==0)

In [ ]:
ml.update_modelgrid()

In [ ]:
import flopy


In [ ]:
array.max()

In [ ]:
array[123,50]

In [ ]:
plt.imshow(out)

In [ ]:
out_folder

In [ ]:
import cartopy.crs as ccrs
from matplotlib.colors import LogNorm
import matplotlib.colorbar as colorbar
import numpy as np

vmin = dict(
    hk = [1.1e-4, 1e2],
           vk = [1, 1e4],
           ss = [1e-7, 1e-2],
           fmp_vk = [1e-6, 1e-1],
)

num = 0
for props in [
'hk',
'vk', 
'ss',
# 'fmp_vk'
]:
# for props in ['fmp_vk']:
    
    
    fig = plt.figure(figsize=(8.5,11))
    fig.suptitle(f"Layer {props}")
    for lay in range(6):
  
        if props=='fmp_vk':
            ax = plt.subplot(1, 1, lay+1, projection=ccrs.epsg(2226))
            filename = f"{props}.txt"
        else:
            ax = plt.subplot(3, 2, lay+1, projection=ccrs.epsg(2226))
            filename = f"{props}{lay+1}.txt"

        Map = flopy.plot.PlotMapView(model = ml, ax = ax)
        
        ibound = ml.bas6.ibound.array[lay]
        array = np.genfromtxt(os.path.join(model_ws,'pp2024_out', filename))
        
        ax.set_title(f"{props} layer {lay+1}")
        # mm = conda_scripts.make_map.make_map(f"Layer {layer+1}, Aquifer Property: {prop}")
        # ax = mm.plotloc(fig, locname = 'SON_MOD')
        array = np.ma.masked_array(array, mask = ibound ==0)
    
        # Set up the normalization using LogNorm
        norm = LogNorm(vmin = vmin[props][0],vmax = vmin[props][1],)
        # c = ax.pcolormesh(sr.xcentergrid,sr.ycentergrid,array,alpha=0.5,  norm = norm )
        c = Map.plot_array(array, norm = norm, alpha = 0.5, cmap = 'PiYG')
        # c = Map.contour_array(array, norm = norm, alpha = 0.5)
        conda_scripts.arich_functions.add_basemaps(ax)
        num+=1
        # ax.scatter(hk_pp2.x, hk_pp2.y,marker='o', c = hk_pp2.zone,s=25, cmap = 'tab10', ec = 'k')
    
        _add_inset(array, ax)

        # Create the inset axes for the histogram
        cax = ax.inset_axes([0.1, 0.005, .8, .03])
        # Add a colorbar at the bottom
        # cbar = plt.colorbar(c, cax=cax, orientation='horizontal')
        # Create a colorbar with the log normalization
        cbar = colorbar.ColorbarBase( cax,  norm=norm, orientation='horizontal', cmap = 'PiYG')
        
        if props=='fmp_vk':
            print('ending')
            break
            
    print(f'{props}.png')
    plt.savefig(os.path.join(out_folder,f'layer_{props}.png'))
    

In [ ]:
import geopandas as gpd
import contextily as ctx
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import conda_scripts
gpd.io.file.fiona.drvsupport.supported_drivers['LIBKML'] = 'rw'

In [ ]:
homes = gpd.read_file(r"C:\Users\arich\Downloads\Untitled project.kml").to_crs(2226)

ax = plt.subplot(projection = ccrs.epsg(2226))
ax = homes.plot(ax = ax)
homes.dissolve().centroid.plot(ax = ax)
ctx.add_basemap(ax, crs = 2226)
conda_scripts.plot_help.remove_ctx_annotations(ax)

# compare calculated arrays versus modflow generated arrays

In [ ]:
arrays_from_mod = dict(hk="PARAM_HKC_G1_L{:}.txt",
ss="PARAM_Ss_G1_L{:}.txt",
vk="PARAM_VKA_G1_L{:}.txt")

arrays_from_mod

In [ ]:
model_ws

In [ ]:
import cartopy.crs as ccrs
from matplotlib.colors import LogNorm
import matplotlib.colorbar as colorbar
import numpy as np

import cartopy.crs as ccrs
import matplotlib.colorbar as colorbar
from matplotlib.colors import LogNorm

os.makedirs(os.path.join(out_folder,'array_explainer'), exist_ok = True)


num = 0
# for props in ['hk','vk', 'ss']: 
for props in ['vk']: 

    fig = plt.figure(figsize=(8.5,11))
    fig.suptitle(f"{props}")
    for lay in range(6):

        # fig = plt.figure(figsize=(8.5,11))
        fig.suptitle(f"{props}", size = 'large')

        if len(props)>3:
            continue

        ax = plt.subplot(3, 2, lay+1, projection=ccrs.epsg(2226))           
        Map = flopy.plot.PlotMapView(model = ml, ax = ax)
            
        ibound = ml.bas6.ibound.array[lay]       
        print(f'{lay} {props}')

        if props=='vk':
            print('using alt\n'*10)
            vk, xxx, mult, hk_arr,zmult =  forward_run.write_pilot_point(lay, props+str(lay+1), model_ws, skip_writing_output = True)
            hk, xxx, mult, hk_arr,zmult =  forward_run.write_pilot_point(lay, 'hk'+str(lay+1), model_ws, skip_writing_output = True)
            out = hk/vk
        else:
            
            out, xxx, mult, hk_arr,zmult =  forward_run.write_pilot_point(lay, props+str(lay+1), model_ws, skip_writing_output = True)
        
        d = {'Final Array':out, "Entire Zone":zmult, "Zone Multiplier":mult, "Pilot Point":hk_arr}


            


        # Set up the normalization using LogNorm
        norm = LogNorm(vmin = 1e-1,vmax = 1e1,)
        
        # ax = plt.subplot(1, 1, 1, projection=ccrs.epsg(2226))
        ax.set_title(lay)
        # mm = conda_scripts.make_map.make_map(f"Layer {layer+1}, Aquifer Property: {prop}")
        # ax = mm.plotloc(fig, locname = 'SON_MOD')
        array = np.ma.masked_array(out, mask = ibound ==0)

        mod_array = np.genfromtxt(os.path.join(model_ws,'output', arrays_from_mod[props].format(lay+1)),skip_header = 1)
        mod_array = np.ma.masked_array(mod_array, mask = ibound ==0)

        plot = array/mod_array

        c = ax.pcolormesh(sr.xcentergrid,sr.ycentergrid,plot,alpha=0.9, norm = norm, cmap = 'PiYG')
        cax = ax.inset_axes([0.1, 0.005, .8, .03])
        # Add a colorbar at the bottom
        # cbar = plt.colorbar(c, cax=cax, orientation='horizontal')
        # Create a colorbar with the log normalization
        # Map.plot_array(plot,)
        cbar = colorbar.ColorbarBase( cax,  norm=norm, orientation='horizontal', cmap = 'PiYG')
        print(f'{lay}_{props}.png')

        ax.text(0,1,f"all values are equal: {np.all(np.isclose(plot.compressed(),1))}",        
            horizontalalignment='left',
            verticalalignment='top',
            transform=ax.transAxes,
                fontsize = 15,
               bbox = {'color':'g' if np.all(np.isclose(plot.compressed(),1)) else 'r'})
    
        num+=1
            
    plt.savefig(os.path.join(out_folder,'array_explainer',f'comp_{props}.png'))
        
    

In [ ]:
vk

In [ ]:
np.genfromtxt(os.path.join(model_ws,'output', arrays_from_mod[prop[0:2]].format(layer+1)),skip_header = 1)

In [ ]:
np.genfromtxt(skip_header = 0)